<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Bi-directional-problem" data-toc-modified-id="Bi-directional-problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bi-directional problem</a></span><ul class="toc-item"><li><span><a href="#Run" data-toc-modified-id="Run-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run</a></span><ul class="toc-item"><li><span><a href="#Init-Graph" data-toc-modified-id="Init-Graph-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Init Graph</a></span></li><li><span><a href="#Run-ICU" data-toc-modified-id="Run-ICU-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Run ICU</a></span></li><li><span><a href="#Run-Outer-Cost-Update" data-toc-modified-id="Run-Outer-Cost-Update-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Run Outer Cost Update</a></span></li></ul></li></ul></li><li><span><a href="#Comparison-of-both-algorithms" data-toc-modified-id="Comparison-of-both-algorithms-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Comparison of both algorithms</a></span></li></ul></div>

# Description

This file aims at implementing both versions of the algorithm (with rebalancers update inside and outside of the loop) and at comparing their performance and convergence to a solution. 

Comparison is done on a simple graph. The dummy problem we talked about. 

# Load

In [12]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from routines_icu import *
from helpers_icu import *
import cvxpy as cp
from FW_icu import *

from FW_OuterUpdate import solve

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bi-directional problem

In [13]:
#parameters
alpha=0.15
beta=4
POTENTIAL_SHIFT=90
INVERSE_DEMAND_SHIFT=80

ZC_EDGE=1

In [14]:
L_dum=10
#to adapt
#to be read from file if necessary
nodes_list=['1','2','2_p','R','1_p']
edge_list=[('1','2'),('2','1'),('2','2_p'),('1','R'),('1','2_p'),('1','1_p'),('2','1_p'),('2','R')]
k_list=[10,10,3,10,ZC_EDGE,2,ZC_EDGE,10] 
l_list=[10,10,1,L_dum,0,1,0,L_dum]
t_list=[10,10,10,10,ZC_EDGE,10,ZC_EDGE,10]
phi_list=[phi(l_list[i],t_list[i]) for i in range(len(l_list))]
is_negative=[0,0,1,0,0,1,0,0]

nodes_pots=[('2_p',POTENTIAL_SHIFT),('1_p',POTENTIAL_SHIFT)]

In [24]:
dummy_nodes=dict()
dummy_nodes['2_p']='2'
dummy_nodes['1_p']='1'

## Run

### Init Graph

In [235]:
#Note: it is the same to write OD[(a,b)] as OD[a,b]

OD=dict()
N=10
OD['1','2_p']=N
OD['2','1_p']=N

In [236]:
G=nx.DiGraph()
G.add_nodes_from(nodes_list)
G.add_edges_from(edge_list)

In [237]:
G=initEdgeAttr(G,edge_list,k_list,phi_list,is_negative)
G=initNodeAttr(G,nodes_pots)
G=update_costs(G,INVERSE_DEMAND_SHIFT)

#Create a feasible solution for the passengers
G=init_flows(G,OD) 

G=update_costs(G,INVERSE_DEMAND_SHIFT)

In [238]:
G_0=G.copy()

### Run ICU

First algorithm with the updates in the loop

In [280]:
G_icu,y_list,opt_res,OD_list=modified_FW(G_0.copy(),OD.copy(),edge_list,dummy_nodes,maxIter=400,
                         step='fixed',rebalancer_smoothing=True, ri_smoothing=False,evolving_bounds=False)

In [281]:
OD_list[-1]

{('1', '2_p'): 10,
 ('2', '1_p'): 10,
 ('1', 'R'): 0,
 ('2', 'R'): 2.8543436630423207}

In [282]:
print_final_flows(G_icu)

('1', '2')  :  8.621499252181273
('1', 'R')  :  2.8227367709211344
('1', '2_p')  :  1.3785007478187314
('1', '1_p')  :  5.7955568419972545
('2', '1')  :  8.621634428101697
('2', '2_p')  :  8.621499252181273
('2', '1_p')  :  4.204443158002741
('2', 'R')  :  0.0


In [283]:
print_final_cost(G_icu)

('1', '2')  :  38.964361480796505
('1', 'R')  :  41.16476111184817
('1', '2_p')  :  90.0
('1', '1_p')  :  50.577903173569744
('2', '1')  :  38.92561497627678
('2', '2_p')  :  50.197055318475336
('2', '1_p')  :  90.0
('2', 'R')  :  10000000000000000000000


### Run Outer Cost Update

Second algorithm with the updates inside the loop

The tolerance value below is on the r_i : how much we need them to be close to each other between two iterations

In [285]:
#tol: difference in ri between two iterations
#FW_tol: value of the certificate

G_FW,ri_FW = solve(G_0.copy(),OD.copy(),edge_list,dummy_nodes,tol=10**-4, FW_tol=10**-4)

##########################################
ITERATION #:  1
CURRENT RI_k
{'1': 0, '2': 0, '2_p': 0, 'R': 0, '1_p': 0}
CURRENT OD: {('1', '2_p'): 10, ('2', '1_p'): 10, ('1', 'R'): 0, ('2', 'R'): 0}
Cost at the beginning of the iteration:
('1', '2')  :  36.0
('1', 'R')  :  10000000000000000000000
('1', '2_p')  :  90.0
('1', '1_p')  :  13.599999999999994
('2', '1')  :  36.0
('2', '2_p')  :  13.599999999999994
('2', '1_p')  :  90.0
('2', 'R')  :  10000000000000000000000
Flows at the beginning of the iteration:
('1', '2')  :  0
('1', 'R')  :  0
('1', '2_p')  :  10
('1', '1_p')  :  0
('2', '1')  :  0
('2', '2_p')  :  0
('2', '1_p')  :  10
('2', 'R')  :  0
##########################################
ITERATION #:  2
CURRENT RI_k
{'1': 2.7939952239025567, '2': -2.7939952239025567, '2_p': 0, 'R': 0.0, '1_p': 0}
CURRENT OD: {('1', '2_p'): 10, ('2', '1_p'): 10, ('1', 'R'): 0, ('2', 'R'): 2.7939952239025567}
Cost at the beginning of the iteration:
('1', '2')  :  41.4
('1', 'R')  :  41.4
('1', '2_p') 

In [286]:
ri_FW

[{'1': 0, '2': 0, '2_p': 0, 'R': 0, '1_p': 0},
 {'1': 2.7939952239025567,
  '2': -2.7939952239025567,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0},
 {'1': 2.8798042972666265,
  '2': -2.8798042972666265,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0},
 {'1': 2.8840491735351117,
  '2': -2.8840491735351117,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0},
 {'1': 2.884269578053429,
  '2': -2.884269578053429,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0},
 {'1': 2.8842795221155475,
  '2': -2.8842795221155475,
  '2_p': 0,
  'R': 0.0,
  '1_p': 0}]

# Comparison of both algorithms

In [291]:
def compare_final(G_icu_list,G_FW_list,att):
    G_icu=G_icu_list[-1]
    G_FW=G_FW_list[-1]
    
    for e in G.edges():
        if att=='flow':
            att_icu=G_icu[e[0]][e[1]]['f_m']+G_icu[e[0]][e[1]]['f_r']
            att_FW=G_FW[e[0]][e[1]]['f_m']+G_FW[e[0]][e[1]]['f_r']
        elif att=='cost':
            att_icu=G_icu[e[0]][e[1]]['cost']
            att_FW=G_FW[e[0]][e[1]]['cost']
        try:
            diff_rel=abs((att_FW-att_icu)/att_FW)
        except:
            if att_FW==0:
                if att_FW-att_icu==0:
                    diff_rel=0.0
            else:
                diff_rel=np.nan
        print(e," : ", np.around(100*diff_rel,2),"%")

In [292]:
compare_final(G_icu,G_FW,'flow')

('1', '2')  :  0.36 %
('1', 'R')  :  2.13 %
('1', '2_p')  :  2.33 %
('1', '1_p')  :  0.47 %
('2', '1')  :  0.36 %
('2', '2_p')  :  0.36 %
('2', '1_p')  :  0.64 %
('2', 'R')  :  0.0 %


In [293]:
compare_final(G_icu,G_FW,'cost')

('1', '2')  :  0.16 %
('1', 'R')  :  0.57 %
('1', '2_p')  :  0.0 %
('1', '1_p')  :  0.78 %
('2', '1')  :  0.26 %
('2', '2_p')  :  1.52 %
('2', '1_p')  :  0.0 %
('2', 'R')  :  0.0 %


In [251]:
print_final_flows(G_icu)

('1', '2')  :  8.621499252181273
('1', 'R')  :  2.825786017029079
('1', '2_p')  :  1.3785007478187314
('1', '1_p')  :  5.7955568419972545
('2', '1')  :  8.621634428101697
('2', '2_p')  :  8.621499252181273
('2', '1_p')  :  4.204443158002741
('2', 'R')  :  2.8075260572391993


In [250]:
print_final_flows(G_FW)

('1', '2')  :  8.65290748974937
('1', 'R')  :  2.8842695780534218
('1', '2_p')  :  1.3470925102504996
('1', '1_p')  :  5.768627967633823
('2', '1')  :  8.652897545687246
('2', '2_p')  :  8.65290748974937
('2', '1_p')  :  4.2313720323661554
('2', 'R')  :  0.0


In [210]:
print_final_cost(G_)

('1', '2')  :  39.032646235968954
('1', 'R')  :  41.400000000000034
('1', '2_p')  :  90.0
('1', '1_p')  :  13.599999999999994
('2', '1')  :  39.032646235968976
('2', '2_p')  :  51.04007698727102
('2', '1_p')  :  90.0
('2', 'R')  :  10000000000000000000000


In [211]:
print_final_cost(G_k)

('1', '2')  :  38.964361480796505
('1', 'R')  :  41.339024064935984
('1', '2_p')  :  90.0
('1', '1_p')  :  13.599999999999994
('2', '1')  :  38.930888385767
('2', '2_p')  :  50.197055318475336
('2', '1_p')  :  90.0
('2', 'R')  :  10000000000000000000000
